In [2]:
import pandas as pd
import polars as pl
import sqlalchemy as sa
import urllib
from datetime import datetime, timedelta
import math
import numpy as np
import datetime as dt
import glob
import numpy as np

In [3]:
def getEngine(engine_name):
    connection = f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(engine_name)}"
    engine = sa.create_engine(connection, fast_executemany=True)
    return engine

In [4]:
### Connect to DataMart Server
mart_server = '118.69.201.34'
mart_database = 'FLC_SHOP_SALES_DATAWAREHOUSE'
mart_username = 'ecom_user'
mart_password = 'Ec0m@12345'
SaleMart = ('DRIVER={ODBC Driver 17 for SQL Server}'
            ';SERVER=' + mart_server +
            ';DATABASE=' + mart_database +
            ';UID=' + mart_username +
            ';PWD=' + mart_password)

In [5]:
### Connect to Azure server
azr_server = "ssd-synapse-prod.sql.azuresynapse.net"
azr_database = 'frtssdsqlpoolprod'
azr_username = 'frtcid'
azr_password = 'Frt2022Cid@#123'
        
connection_url = sa.engine.URL.create(
    "mssql+pyodbc",
    username=azr_username,
    password=azr_password,
    host=azr_server,
    database=azr_database,
    query={
        "driver": "ODBC Driver 17 for SQL Server",
        "autocommit": "True",
    },
)

azr_engine = sa.create_engine(connection_url)

In [19]:
source_ = 'Data_20230629_20230629'
df = pd.read_excel(r"D:/FRT/31.Data_Akachain/1.Data_Template/2023-06/" + source_ + ".xlsx")
df

,Mã thành viên,Ngày tạo,Ngày giao dịch,Lý do,Mã giao dịch,Số tiền mua,Điểm xếp hạng quy đổi,Điểm phát sinh,Điểm đổi quà đã sử dụng
0,3192ff73-d1a3-4501-94b6-b65ac415166b,2023-06-29,2023-06-29,PartnerRefund,45294626,NaN,0,3122,NaN
1,0e45ba44-d594-4bb3-b273-2304e0e78018,2023-06-29,2023-06-29,PartnerRefund,45289727,NaN,0,7000,NaN
2,cb718540-7862-49f4-a3bb-96c16bb30256,2023-06-29,2023-06-29,PartnerRefund,45294320,NaN,0,2900,NaN
3,ebeed53a-e34d-433f-8b86-4946f3c42803,2023-06-29,2023-06-29,PartnerRefund,45290910,NaN,0,647,NaN
4,2f21bb47-bf93-441f-90a5-6e279d1dd814,2023-06-29,2023-06-29,PartnerRefund,45289931,NaN,0,3676,NaN
...,...,...,...,...,...,...,...,...,...
8622,0119d61e-6cbd-4b6c-b3e1-e7b2409c0817,2023-06-29,2023-06-29,Invoice,45285878,75000.0,18,18,NaN
8623,57376206-1ff7-4c3e-96eb-6ee73cd11439,2023-06-29,2023-06-29,Invoice,45285877,135000.0,33,33,NaN
8624,b7a00cfb-4afd-420f-80bb-67ef21aa1114,2023-06-29,2023-06-29,Invoice,45285876,1390000.0,347,347,NaN
8625,b681b476-d2d8-4432-8662-83b9a9973027,2023-06-29,2023-06-29,Invoice,45285875,135000.0,33,33,NaN


In [10]:
df['Ngày tạo']          = pd.to_datetime(df['Ngày tạo'])
df['Ngày giao dịch']    = pd.to_datetime(df['Ngày giao dịch'])

col_ = ['Mã thành viên', 'Ngày tạo', 'Ngày giao dịch', 'Lý do', 'Mã giao dịch',
       'Số tiền mua', 'Điểm xếp hạng quy đổi', 'Điểm phát sinh',
       'Điểm đổi quà đã sử dụng']
df = df[col_]
df['TIME_INSERT']   = datetime.now()
df['Source']        = source_
min_date_filter_    = df['Ngày tạo'].min() - timedelta(days=5)


In [17]:
df.sample(10)

,Mã thành viên,Ngày tạo,Ngày giao dịch,Lý do,Mã giao dịch,Số tiền mua,Điểm xếp hạng quy đổi,Điểm phát sinh,Điểm đổi quà đã sử dụng,TIME_INSERT,Source
5257,c8a72dd9-8084-486a-b1ad-650d0cb04cda,2023-06-29,2023-06-29,Invoice,45290794,315000.0,78,78,NaN,2023-07-03 11:52:19.688118,Data_20230629_20230629
2181,458d52f8-0fe4-4901-b7df-8e0a7ab353f9,2023-06-29,2023-06-29,Invoice,45294560,520000.0,130,130,NaN,2023-07-03 11:52:19.688118,Data_20230629_20230629
8047,a68def1e-e728-439c-9764-5fe13cbaa096,2023-06-29,2023-06-29,InvoiceReturnFull,1102683,225000.0,-56,-56,NaN,2023-07-03 11:52:19.688118,Data_20230629_20230629
6303,595a6df9-c115-4333-90b0-e1f030c90215,2023-06-29,2023-06-29,Invoice,45289355,1590000.0,397,397,NaN,2023-07-03 11:52:19.688118,Data_20230629_20230629
7948,37a3aa2d-da85-486e-9410-79bd126f0220,2023-06-29,2023-06-29,Invoice,45287011,99000.0,24,24,NaN,2023-07-03 11:52:19.688118,Data_20230629_20230629
6332,39a0b81d-5d3d-4b12-8080-d709e77de4cc,2023-06-29,2023-06-29,Invoice,45289198,12040000.0,3010,3010,NaN,2023-07-03 11:52:19.688118,Data_20230629_20230629
6617,17813463-4105-4c67-b6a6-5c77653edfa1,2023-06-29,2023-06-29,Invoice,45288849,0.0,0,0,NaN,2023-07-03 11:52:19.688118,Data_20230629_20230629
1211,8f25a2e9-bae7-4a62-92ea-ba8de6cbb7f9,2023-06-29,2023-06-29,Invoice,45295800,220000.0,55,55,NaN,2023-07-03 11:52:19.688118,Data_20230629_20230629
1331,49a8b3a1-392f-4879-81a1-9c97d82efc0f,2023-06-29,2023-06-29,Invoice,45295107,351000.0,87,87,NaN,2023-07-03 11:52:19.688118,Data_20230629_20230629
1403,3e2c63f8-1e66-4db2-920e-08ada30b13be,2023-06-29,2023-06-29,Invoice,45295566,340000.0,85,85,NaN,2023-07-03 11:52:19.688118,Data_20230629_20230629


In [15]:
## Lấy mã giao dịch dò trong Server Azure để tìm ra mã khách hàng
list_inv = df[df['Lý do']=='Invoice']['Mã giao dịch'].drop_duplicates()
num_part = math.ceil(len(list_inv) /15_000)
list_cus_inv_out = pd.DataFrame()
print(f"Number of records = {len(list_inv):,.0f}")

for i in range(1,num_part+1):
    list_ = list_inv[(i-1)*15_000 : min(i * 15_000, len(list_inv))]
    num_record = min(i * 15_000, len(list_inv))
    print(f"|_{i}_{num_record:,.0f}_{num_record/len(list_inv):,.0%}")
    list_ = ','.join(list_)

    query_read = f"""SELECT DISTINCT a.CUSTOMER_KEY, a.INVOICE_HEADER 
                FROM VF_CID_SALES_TRANSACTION_ICT a
                LEFT JOIN (SELECT value as 'INVOICE_HEADER' FROM STRING_SPLIT( '{list_}' , ',')) b
                    ON a.INVOICE_HEADER = b.INVOICE_HEADER
                WHERE b.INVOICE_HEADER IS NOT NULL"""      
     
    list_cus_inv = pd.read_sql(query_read, azr_engine)
    list_cus_inv_out = pd.concat([list_cus_inv_out, list_cus_inv])
list_cus_inv_out['INVOICE_HEADER']=list_cus_inv_out['INVOICE_HEADER'].apply(str)


Number of records = 8,303
|_1_8,303_100%


C:\Users\123\AppData\Local\Temp\ipykernel_20560\2374512287.py:7: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  list_ = list_inv[(i-1)*15_000 : min(i * 15_000, len(list_inv))]


In [ ]:
# Dòng đầu tiên tạo một Series list_inv chứa các giá trị duy nhất từ cột 'Mã giao dịch' trong df khi cột 'Lý do' bằng 'Invoice'.
# Các giá trị trong list_inv được loại bỏ các bản sao.
# Dòng thứ hai tính toán số lượng phần tử trong list_inv chia cho 15,000 và làm tròn lên bằng hàm math.ceil(),
# kết quả được gán vào biến num_part.
# Dòng thứ tư tạo một DataFrame rỗng list_cus_inv_out để lưu trữ kết quả cuối cùng.
# Dòng thứ năm in ra số lượng bản ghi trong list_inv dưới dạng số nguyên với định dạng dấu phẩy ngăn cách hàng nghìn.
# Vòng lặp for từ 1 đến num_part+1 được sử dụng để chia list_inv thành các phần nhỏ với mỗi phần chứa tối đa 15,000 phần tử. 
# Mỗi phần được gán vào biến list_.
# Dòng thứ 8 tính toán số lượng bản ghi trong list_ và in ra thông tin về phần hiện tại của vòng lặp, bao gồm số thứ tự phần,
# số lượng bản ghi và phần trăm so với tổng số bản ghi ban đầu.
# Dòng thứ 9 chuyển đổi list_ thành một chuỗi các giá trị được ngăn cách bởi dấu phẩy.
# Dòng thứ 10 tạo một câu truy vấn SQL để truy xuất dữ liệu từ cơ sở dữ liệu. 
# Câu truy vấn này sẽ lấy các cặp giá trị CUSTOMER_KEY và INVOICE_HEADER từ bảng 
# VF_CID_SALES_TRANSACTION_ICT chỉ khi INVOICE_HEADER có giá trị nằm trong list_.
# Dòng thứ 11 sử dụng pd.read_sql() để thực thi câu truy vấn SQL và trả về kết quả dưới dạng DataFrame, được gán vào biến list_cus_inv.
# Dòng thứ 12 sử dụng pd.concat() để kết hợp list_cus_inv với list_cus_inv_out để tạo một DataFrame tổng hợp kết quả từ tất cả các phần.
# Dòng cuối cùng áp dụng phương thức .apply() lên cột 'INVOICE_HEADER' trong list_cus_inv_out 
# để chuyển đổi các giá trị trong cột này thành kiểu dữ liệu chuỗi (string).
# Tóm lại, đoạn mã này thực hiện việc trích xuất dữ liệu từ cơ sở dữ liệu SQL Server dựa trên các giá trị 
# trong cột 'Mã giao dịch' của DataFrame df. Sau đó, nó chia các giá trị thành các phần nhỏ và truy vấn cơ sở dữ liệu 
# để lấy các cặp CUSTOMER_KEY và INVOICE_HEADER tương ứng. Cuối cùng, kết quả được lưu trữ trong một DataFrame tổng hợp.